In [1]:
import time
from datetime import timedelta, datetime
import os,sys
import requests,json

In [3]:
def getDate(postdays=1):
    """
    Function to get the next date => Today + 1 day
    Returns
    -------
    date : String
        Next date in DD-MM-YYYY format
    """
    tomorrow = (datetime.today() + timedelta(postdays)).strftime("%d-%m-%Y")
    return tomorrow

In [4]:
def getStateID(state):
  state_id=''
  url = "https://cdn-api.co-vin.in/api/v2/admin/location/states"
  try:
      response = requests.get(url,headers=headers)
      parsed_response = json.loads(response.text)
      state_length = len(parsed_response['states'])
      for idx in range(state_length):
          if((parsed_response['states'][idx]['state_name']).lower().replace(" ", "") == state.lower().replace(" ", "")):
              state_id = parsed_response['states'][idx]['state_id']
              return(state_id)
  except Exception as e:
      print(e)

In [5]:
def pincodeToStateDistrictConverter(pincode):
    district = ''
    state = ''
    india_post_url = "https://api.postalpincode.in/pincode/{pin}".format(pin = pincode)
    try:
        response = requests.get(india_post_url,headers=headers)
        parsed_response = json.loads(response.text)
        if(parsed_response[0]["Status"] == "Success"):
            district = parsed_response[0]['PostOffice'][0]['District']
            state = parsed_response[0]['PostOffice'][0]['State']
        return district,state
    except Exception as e:
        print(e)

In [7]:
def checkAvailability(payload, age):
    """
    Function to check availability in the hospitals based on
    user age from the json response from the public API
    Parameters
    ----------
    payload : JSON
    age: INT
    Returns
    -------
    available_centers_str : String
        Available hospitals
    total_available_centers : Integer
        Total available hospitals
    """
    available_centers = dict()
    unavailable_centers = set()
    available_centers_str = False
    total_available_centers = 0
    
    if('centers' in payload.keys()):
       length = len(payload['centers'])
       if(length>1):
            for i in range(length):
                sessions_len = len(payload['centers'][i]['sessions'])
                for j in range(sessions_len):
                    if((payload['centers'][i]['sessions'][j]['available_capacity']>0) and
                       (payload['centers'][i]['sessions'][j]['min_age_limit']<=age)):
                        available_centers[total_available_centers]=payload['centers'][i]
                        total_available_centers +=1
                        break
    
    return available_centers,total_available_centers

In [8]:
def getDistrictID(st_id, lookout_district):
    districtid = ''
    url = "https://cdn-api.co-vin.in/api/v2/admin/location/districts/{st}".format(st = st_id)
    try:
        response = requests.get(url,headers=headers)
        parsed_response = json.loads(response.text)
        district_length = len(parsed_response['districts'])
        for idx in range(district_length):
            if((parsed_response['districts'][idx]['district_name']).lower().replace(" ", "") == lookout_district.lower().replace(" ", "")):
                district_id = parsed_response['districts'][idx]['district_id']
        return (district_id)
    except Exception as e:
        print(e)

In [9]:
def pingCOWIN(date,district_id):
    """
    Function to ping the COWIN API to get the latest district wise details
    Parameters
    ----------
    date : String
    district_id : String
    
    Returns
    -------
    json
    """
    url = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={district_id}&date={date}".format(district_id = district_id, date = date)
    response = requests.get(url,headers=headers)
    return json.loads(response.text)

In [10]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36 Edg/91.0.864.54'}
settings = {
  "daysLeftforDose2":5,
  "pincode":700047,
  "userAge": 19
}

In [11]:
dist,st=pincodeToStateDistrictConverter(settings["pincode"])

In [13]:
stateid=getStateID(st)

In [14]:
distid=getDistrictID(stateid,dist)

In [15]:
centers=pingCOWIN(getDate(settings["daysLeftforDose2"]),distid)
len(centers["centers"])

6

In [16]:
available,count=checkAvailability(centers,settings["userAge"])

In [24]:
for i in range(len(available.keys())):
    print("A "+available[i]['fee_type']+" Center Named "+available[i]['name']+" at address "+available[i]['address']+" is available from "+available[i]['from']+" to "+available[i]['to'])
    print("Vaccine - "+available[i]['vaccine_fees'][0]["vaccine"]+" priced at Rs."+available[i]['vaccine_fees'][0]["fee"]+"(inclusive of all service charges and 5% GST,if applicable)")
    for j in range(len(available[i]['sessions'])):
        if available[i]['sessions'][j]['min_age_limit'] < settings["userAge"]:
            print("Session ID - "+available[i]['sessions'][j]['session_id'])
            print("Date - "+available[i]['sessions'][j]['date'])
            print("Min. Age - "+str(available[i]['sessions'][j]['min_age_limit']))
            # print("Vaccine Name - "+available[i]['sessions'][j]['vaccine'])
            print("Quantity of 1st Dose Available is "+str(available[i]['sessions'][j]['available_capacity_dose1']))
            print("Quantity of 2nd Dose Available is "+str(available[i]['sessions'][j]['available_capacity_dose2']))
            print("Time Slots "+str(available[i]['sessions'][j]['slots']))
    print("\n ---------------- \n")

A Paid Center Named Apollo Clinic COVAXIN at address 6 Vivekananda Road. Ganesh Talkies Kol-700007 is available from 15:30:00 to 18:00:00
Vaccine - COVAXIN priced at Rs.1410(inclusive of all service charges and 5% GST,if applicable)
Session ID - d7638340-6107-46c5-b6f3-765b747f0ebf
Date - 12-07-2021
Min. Age - 18
Quantity of 1st Dose Available is 82
Quantity of 2nd Dose Available is 57
Time Slots ['03:30PM-05:30PM', '05:30PM-06:00PM']
Session ID - bf957855-71bd-441c-97a7-2f25225f5fcf
Date - 13-07-2021
Min. Age - 18
Quantity of 1st Dose Available is 92
Quantity of 2nd Dose Available is 29
Time Slots ['03:30PM-05:30PM', '05:30PM-06:00PM']
Session ID - 9c2492b9-8831-4ba3-9c2d-a7934529f753
Date - 14-07-2021
Min. Age - 18
Quantity of 1st Dose Available is 92
Quantity of 2nd Dose Available is 22
Time Slots ['03:30PM-05:30PM', '05:30PM-06:00PM']
Session ID - b0bafd98-46c8-467b-92e0-4cc077a2f518
Date - 15-07-2021
Min. Age - 18
Quantity of 1st Dose Available is 60
Quantity of 2nd Dose Available

In [19]:
def getAvailableNames(payload, age):
    """
    Function to check availability in the hospitals based on
    user age from the json response from the public API
    Parameters
    ----------
    payload : JSON
    age: INT
    Returns
    -------
    available_centers_str : String
        Available hospitals
    total_available_centers : Integer
        Total available hospitals
    """
    available_centers = set()
    unavailable_centers = set()
    available_centers_str = False
    total_available_centers = 0
    
    if('centers' in payload.keys()):
       length = len(payload['centers'])
       if(length>1):
            for i in range(length):
                sessions_len = len(payload['centers'][i]['sessions'])
                for j in range(sessions_len):
                    if((payload['centers'][i]['sessions'][j]['available_capacity']>0) and
                       (payload['centers'][i]['sessions'][j]['min_age_limit']<=age)):
                        available_centers.add(payload['centers'][i]['name'])
            available_centers_str =  ", ".join(available_centers)
            total_available_centers = len(available_centers)
    
    return available_centers_str,total_available_centers

In [ ]:
getAvailableNames(centers,settings['userAge'])

In [23]:
time.sleep(5)

In [19]:
for i in range(1,31):
    settings['daysLeftforDose2']=i
    print(getAvailableNames(pingCOWIN(getDate(settings['daysLeftforDose2']),distid),settings['userAge']))
    # time.sleep(2)
    # print("",end="\r")


('Medica Hospital COVAXIN, SURAKSHA Kalikapur, AMRI MUKUNDAPUR-COVAXIN, SURAKSHA ELGIN, Woodlands MultispecialityHosp, Fortis Hospital., SURAKSHA DIAG PVT.LTD-KASBA, Apollo Clinic Thakurpukur, AMRI Hospitals Dhakuria, Apollo Gleneagles Med-Gariahat, Peerless Hospital, SURAKSHA JADAVPUR, AMRI Hospital Mukundapur, Woodlands Hospital COVAXIN, Apollo Clinic COVAXIN, Narayan Memorial Hospital, FORTIS MEDICAL Centre, IQCityMCH-Kol, SURAKSHA DIAGNOSTIC- Behala, Apollo Clinic, AMRI Med Centre Southern Avenu, Suraksha Diagonistic- Garia, MEDICA SuperSpecialityHospital', 23)
('Medica Hospital COVAXIN, SURAKSHA Kalikapur, SURAKSHA ELGIN, Woodlands MultispecialityHosp, Fortis Hospital., SURAKSHA DIAG PVT.LTD-KASBA, AMRI Hospitals Dhakuria, Peerless Hospital, SURAKSHA JADAVPUR, AMRI Hospital Mukundapur, Woodlands Hospital COVAXIN, Apollo Clinic COVAXIN, FORTIS MEDICAL Centre, SURAKSHA DIAGNOSTIC- Behala, IQCityMCH-Kol, Apollo Clinic, AMRI Med Centre Southern Avenu, Suraksha Diagonistic- Garia, MEDI

In [22]:
for i in range(1,31):
    centers = pingCOWIN(getDate(i),distid)
    print("For Date - "+getDate(i))
    for j in range(len(centers['centers'])):
        center = centers['centers'][j]
        if len(centers['centers'])==0:
            print("No Data Available")
        print("["+center['fee_type']+"]"+center['name']+" at",center['address'])
        try:
            print("Vaccine - "+center['vaccine_fees'][0]['vaccine'],"Fees - "+center['vaccine_fees'][0]['fee'])
        except:
            print("Free of Cost Vaccine")
        print("Session Details")
        for k in range(len(center['sessions'])):
            sessions = center['sessions'][k]
            print(sessions['date'],sessions['vaccine'],"Minimum Age : "+str(sessions['min_age_limit']),"Available Doses for Dose 1: "+str(sessions['available_capacity_dose1']),"Available Doses for Dose 2: "+str(sessions['available_capacity_dose2']))
            print(sessions['slots'])
        print("\n\n")

For Date - 24-06-2021
[Paid]IQCityMCH-Kol at Mani Square Mall 164-1 Maniktalla Main Road
Vaccine - COVAXIN Fees - 1410
Session Details
24-06-2021 COVAXIN Minimum Age : 45 Available Doses for Dose 1: 295 Available Doses for Dose 2: 297
['10:00AM-11:00AM', '11:00AM-12:00PM', '12:00PM-01:00PM', '01:00PM-05:30PM']
24-06-2021 COVAXIN Minimum Age : 18 Available Doses for Dose 1: 272 Available Doses for Dose 2: 283
['10:00AM-11:00AM', '11:00AM-12:00PM', '12:00PM-01:00PM', '01:00PM-05:30PM']
25-06-2021 COVAXIN Minimum Age : 18 Available Doses for Dose 1: 273 Available Doses for Dose 2: 168
['10:00AM-11:00AM', '11:00AM-12:00PM', '12:00PM-01:00PM', '01:00PM-05:30PM']
25-06-2021 COVAXIN Minimum Age : 45 Available Doses for Dose 1: 293 Available Doses for Dose 2: 292
['10:00AM-11:00AM', '11:00AM-12:00PM', '12:00PM-01:00PM', '01:00PM-05:30PM']



[Paid]SURAKSHA Kalikapur at 90 KALIKATALA ROAD
Vaccine - COVISHIELD Fees - 780
Session Details
24-06-2021 COVISHIELD Minimum Age : 45 Available Doses for 

In [21]:
pingCOWIN(getDate(5),distid)['centers']

[{'center_id': 674084,
  'name': 'SURAKSHA JADAVPUR',
  'address': '56Ek Ta Heights Building Raja S.C Mullick Road',
  'state_name': 'West Bengal',
  'district_name': 'Kolkata',
  'block_name': 'Borough-V',
  'pincode': 700032,
  'lat': 22,
  'long': 88,
  'from': '09:00:00',
  'to': '16:00:00',
  'fee_type': 'Paid',
  'sessions': [{'session_id': 'cea098c7-f409-4bc7-ad3e-67e0211afbf8',
    'date': '28-06-2021',
    'available_capacity': 141,
    'min_age_limit': 18,
    'vaccine': 'COVISHIELD',
    'slots': ['09:00AM-11:00AM',
     '11:00AM-01:00PM',
     '01:00PM-03:00PM',
     '03:00PM-04:00PM'],
    'available_capacity_dose1': 141,
    'available_capacity_dose2': 0},
   {'session_id': 'b59064eb-7550-4c4d-b458-a176fb4b7fed',
    'date': '28-06-2021',
    'available_capacity': 35,
    'min_age_limit': 45,
    'vaccine': 'COVISHIELD',
    'slots': ['09:00AM-11:00AM',
     '11:00AM-01:00PM',
     '01:00PM-03:00PM',
     '03:00PM-04:00PM'],
    'available_capacity_dose1': 35,
    'availa